<a href="https://colab.research.google.com/github/eyaJLIDI/churn_prediction_NL/blob/main/preparation_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
import sklearn 
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
def modelisation(data):
  spark =  SparkSession \
        .builder \
        .appName("NL+ churn prediction") \
        .getOrCreate()

  df = spark.createDataFrame(data)
  display(df)
  df = df.withColumn("evol_trajet",df.evol_trajet.cast('double'))
  categoricalColumns = ['PROFIL_IMPAYE', 'SEXE', 'CLT_SOLLICITATION_IND', 'PROFIL_TARIFAIRE17', 'MEDIA', 'CLT_SOLLICITATION_IDFM_IND', 'TYPE_RESIL', 'ACTEUR_RESIL', 'profil_tarifaire46']
  stages = []

  for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index', handleInvalid = 'error')
    #stringIndexer = stringIndexer.setHandleInvalid("error")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

  label_stringIdx = StringIndexer(inputCol = 'CHURN_shift', outputCol = 'label')
  stages += [label_stringIdx]
  numericCols = ['AGE', 'NOMBRE_CONTRAT_IR_SCO','NOMBRE_CONTRAT_IR_ETU','NOMBRE_CONTRAT_NA','NOMBRE_CONTRAT_NMS','NOMBRE_GRAT','NOMBRE_SOL','NOMBRE_AME','ADR_CODE_POSTAL','PORTEUR_PAYEUR','CLT_SEL_IND','MONTANT23','ANCIENNETE_NLP',
 'CONTRATS_PRECEDENTS','NOMBRE_DEPT_RESIDENCE_MOIS','DT_MEAN','ECART_TYPE','VOL_INTERACTION','VOL_EVT','performance','VOL_EVT_SIMPLE','VOL_INTERACTION_SIMPLE','DT_MEAN_SIMPLE','VOL_EVT_STANDARD','VOL_INTERACTION_STANDARD','DT_MEAN_STANDARD','VOL_EVT_COMPLIQUE',
 'VOL_INTERACTION_COMPLIQUE','DT_MEAN_COMPLIQUE','VOL_EVT_BLOQ','VOL_INTERACTION_BLOQ','DT_MEAN_BLOQ','nombre','montant48','montant_offert','evol_trajet']
  assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
  assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
  stages += [assembler]

  pipeline = Pipeline(stages = stages)
  pipelineModel = pipeline.fit(df)
  df = pipelineModel.transform(df)
  selectedCols = ['label', 'features']
  df = df.select(selectedCols)
  df.printSchema()
  classification_data = df.select('features', 'label')
  classification_data.show()
  train, test = classification_data.randomSplit([0.7, 0.3])
  dtc = DecisionTreeClassifier(labelCol="label")
  dtc = dtc.fit(train)
  pred = dtc.transform(test)
  pred.show(10)
  evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
  acc = evaluator.evaluate(pred)
 
  print("Prediction Accuracy: ", acc)
 
  y_pred=pred.select("prediction").collect()
  y_orig=pred.select("label").collect()

  print(classification_report(y_orig, y_pred))

  print('Random forest classifier :')
  rfc = RandomForestClassifier(featuresCol="features", labelCol="label")
  rfc = rfc.fit(train)

  pred = rfc.transform(test)
  pred.show(3)
  evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
  acc = evaluator.evaluate(pred)
 
  print("Prediction Accuracy: ", acc)
 
  y_pred=pred.select("prediction").collect()
  y_orig=pred.select("label").collect()

  cm = confusion_matrix(y_orig, y_pred)
  print("Confusion Matrix:")
  print(cm)